In [ ]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from matplotlib import pyplot
from xgboost import XGBRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
train_df = pd.read_excel('Final_Train.xlsx')
test_df = pd.read_excel('Final_Test.xlsx')
sub_df = pd.read_excel('Sample_submission.xlsx')

In [ ]:
print("Lookup on train data",train_df.head())
print("Shape of train data",train_df.shape)
print("Null value in train data set", train_df.isnull().sum())
print("Lookup into test data",test_df.head())
print("Sum of Null value in test ",test_df.isnull().sum())

Extract String from exp

In [ ]:
train_df['Experience'] = train_df['Experience'].str[0:-17].astype(int)
test_df['Experience'] = test_df['Experience'].str[0:-17].astype(int)

xtarct Rating string

In [ ]:
train_df['Rating'] = train_df['Rating'].str[0:-1].astype(float)
test_df['Rating'] = test_df['Rating'].str[0:-1].astype(float)

ill the null value in place

In [ ]:
train_df['Place'].fillna('none, none', inplace=True)
test_df['Place'].fillna('none, none', inplace=True)

Extract the Area and City from place

In [ ]:
train_df['Area'] = train_df['Place'].str.rsplit(',', 1).str.get(0)
train_df['City'] = train_df['Place'].str.rsplit(',', 1).str.get(1)
test_df['Area'] = test_df['Place'].str.rsplit(',', 1).str.get(0)
test_df['City'] = test_df['Place'].str.split(',', 1).str.get(1)

In [ ]:
train_df['City'] = train_df['City'].str.strip()
test_df['City'] = test_df['City'].str.strip()

In [ ]:
print("Lookup data after extraction",train_df.head())
print("Unique value for city After Extraction",train_df['City'].unique())
print("Unique value for city After Extraction",test_df['City'].unique())

In [ ]:
print("Null vaule",train_df[train_df['City'].isnull()])
train_df.loc[train_df['City'].isnull(), 'Area'] = 'none'
train_df['City'] = train_df['City'].fillna('none')

In [ ]:
train_df['City'].unique()
train_df['Area'].nunique()

In [ ]:
train_df['Profile'].unique()
test_df['Profile'].unique()

bitwise operator to check Misc info present or not 1 or 0

In [ ]:
train_df['Has_M_Info'] = (~train_df['Miscellaneous_Info'].isnull())
test_df['Has_M_Info'] = (~test_df['Miscellaneous_Info'].isnull())
train_df['Has_M_Info'] = train_df['Has_M_Info'].astype(int)
test_df['Has_M_Info'] = test_df['Has_M_Info'].astype(int)

In [ ]:
print("Look up after Misc info",train_df.head())

In [ ]:
quals = pd.DataFrame(item for item in train_df['Qualification'].str.split(', '))
print("Qualification is",quals.head())

In [ ]:
print("Null for Qualification",quals.isnull().sum())

In [ ]:
for i in range(10):
    print(quals[i].nunique())

In [ ]:
print("value count for Qualifivation", quals[0].value_counts())

In [ ]:
vecz = TfidfVectorizer(analyzer='word')
vecz.fit(train_df['Qualification'])

In [ ]:
qual_train = vecz.transform(train_df['Qualification'])
qual_test = vecz.transform(test_df['Qualification'])

ingular Value Decomposition (SVD) is a matrix factorization technique that<br>
factors a matrix M into the three matrices U, Î£, and V. This is very similar to PCA,<br>
excepting that the factorization for SVD is done on the data matrix, whereas for PCA,<br>
the factorization is done on the covariance matrix. Typically, SVD is used under the hood to<br>
find the principle components of a matrix.

In [ ]:
svd = TruncatedSVD(n_components=20)
svd.fit(qual_train)

In [ ]:
print("Variance sum ",svd.explained_variance_ratio_.sum())
print("Variance ratio",svd.explained_variance_ratio_)

In [ ]:
qual_train = svd.transform(qual_train)
qual_train = pd.DataFrame(qual_train, columns=['svd_{}'.format(i) for i in range(20)])
train_df = pd.concat((train_df, qual_train), axis=1)

In [ ]:
qual_test = svd.transform(qual_test)
qual_test = pd.DataFrame(qual_test, columns=['svd_{}'.format(i) for i in range(20)])
test_df = pd.concat((test_df, qual_test), axis=1)

In [ ]:
train_df.to_csv("train.csv")
test_df.to_csv("test.csv")
Y=train_df["Fees"].values
X=train_df.drop(['Fees'],axis=1)
array = X.values
X = array[:,8:30]
#Y = array[:,6]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y,test_size=validation_size, random_state=seed)

Test options and evaluation metric

In [ ]:
num_folds = 10
seed = 7
scoring = 'neg_mean_squared_error'

pot-Check Algorithms

In [ ]:
models = []
models.append(('LR', LinearRegression()))
models.append(('XGB', XGBRegressor()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVR', SVR()))

evaluate each model in turn

In [ ]:
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

owest negtive MSE is for LR followed by LASSO

R: -31072.127725 (1852.614533)<br>
GB: -32960.497059 (2147.411868)<br>
ASSO: -31285.753686 (1870.626560)<br>
N: -35096.030674 (1803.986785)<br>
NN: -35964.710138 (3502.354681)<br>
ART: -39118.208803 (4076.907318)<br>
VR: -33103.136033 (2038.209706)

ompare Algorithms

In [ ]:
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

valuate Algorithms: Standardization

Standardize the dataset

In [ ]:
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('XGB',XGBRegressor())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())])))
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

After Scaling below output gives lowest MSE for LR, LASSO and EN<br>
caledLR: -31073.285058 (1852.428878)<br>
caledLR: -33000.053603 (2380.501922)<br>
caledLASSO: -31057.987851 (1826.336022)<br>
caledEN: -31605.583914 (1711.448216)<br>
caledKNN: -35694.351220 (2920.487089)<br>
caledCART: -38696.891647 (4262.623551)<br>
caledSVR: -34017.125788 (2013.529279)

Compare Algorithms

In [ ]:
fig = pyplot.figure()
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

Algorithm tuning

In [ ]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
k_values = np.array([1,0.1,0.01,0.001,0.0001,0])
param_grid = dict(alpha=k_values)
model = Lasso()
kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("Best Estimator",grid.best_estimator_.alpha)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("mean, Standard Deviation and params %f (%f) with: %r" % (mean, stdev, param))

valuate four different ensemble machine learning<br>
lgorithms, two boosting and two bagging methods:<br>
 Boosting Methods: AdaBoost (AB) and Gradient Boosting (GBM).<br>
 Bagging Methods: Random Forests (RF) and Extra Trees (ET).

ensembles

In [ ]:
ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB',AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor())])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesRegressor())])))

In [ ]:
results = []
names = []
for name, model in ensembles:
    kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

Output is best for GBM<br>
caledAB: -32203.544297 (2290.880793)<br>
caledGBM: -30424.137942 (2154.809940)<br>
caledRF: -31673.784577 (2543.114782)<br>
caledET: -32788.928779 (2697.184527)

Compare Algorithms

In [ ]:
fig = pyplot.figure()
fig.suptitle('Scaled Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

Tune Ensemble Methods<br>
GBM<br>
Tune scaled GBM

In [ ]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = dict(n_estimators=np.array([50,100,150,200,250,300,350,400]))
model = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed,shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

##Finalize the Model<br>
 prepare the model

In [ ]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = GradientBoostingRegressor(random_state=seed, n_estimators=400)
model.fit(rescaledX, Y_train)

transform the validation dataset

In [ ]:
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(mean_squared_error(Y_validation, predictions))

n final test data

In [ ]:
final_df=test_df.values
Y_final=final_df[:,8:29]
final_prediction=model.predict(Y_final)
print(final_prediction)
final_prediction_res=pd.DataFrame(final_prediction,columns=['Fees']).round(decimals=0)
final_prediction_res.to_excel("Final_Submission.xlsx",index=False)